# Sistema de controle do CTA pulsado

In [ ]:
include("../src/ThermistorHW.jl")
using ThermistorHW
using Plots
using CurveFit
using DifferentialEquations
gr()

In [ ]:
include("cta-pulsed.jl")

In [ ]:
Rt = Thermistor(5000, 3000)
Tw = 85.0
cta = CTA.PulsedCTA(Rt)
Rw = Rt(Tw)
htrans = CTA.HeatTrans(Tw)
Ei = cta.Vi
Ri = cta.Ri
mcp = htrans.mcp
Ta = htrans.Ta

In [ ]:
u = 0.2:0.1:20.0
β₁ = CTA.hamcpfun.(htrans, u, 0.0, 25);
β = CTA.hamcpfun.(htrans, u, 0.0, Tw);
η = Rw / (Rw + Ri)^2 * Ei^2/mcp


In [ ]:
plot(u, β₁)
plot!(u, β)

In [ ]:
usqrt = sqrt.(u)
a1,a2, na = king_fit(sqrt.(β), u)
betafun(u) = a1 + a2 * u^na
plot(u, β)
plot!(u, betafun.(u))


In [ ]:
xᵤ = @. sqrt( β * (Tw-Ta) / η )
b1,b2,nb = king_fit(sqrt.(xᵤ), u)
xfun(u) = b1 + b2*u^nb
plot(u, xᵤ)
plot!(u, xfun.(u))


In [ ]:
function efun(x, R, Ri=100.0, Ei=24.0)
        
    Eim = x * Ei
    
    I = Eim / (R + Ri)
    
    return I * Ri
end


In [ ]:
function controle(du, u, p, t)
    Tw = p[1]
    β = p[2]
    η = p[3]
    α₁ = p[4]
    α₂ = p[5]
    dT = -β*u[1] + η*u[2]^2
    dx = -α₁*dT - α₂*(u[1]-Tw)
    du[1] = dT
    du[2] = dx
end

In [ ]:
u0 = [0.0, 0.5]
tspan = (0.0, 200.0)
p = (50.0, 0.22, η, 0.00015, 0.00012)
prob = ODEProblem(controle, u0, tspan, p)

In [ ]:
s = solve(prob);

plot(s.t, s[1,:]./50.0)
plot!(s.t, s[2,:])

In [ ]:
function controle2(du, u, p, t)
    Tw = p[1]
    vel = p[2]
    η = p[3]
    α₁ = p[4]
    α₂ = p[5]
    V= vel(t)
    β = betafun(V)
    dT = -β*(u[1]-20.0) + η*u[2]^2
    dx = -α₁*dT - α₂*(u[1]-Tw)
    du[1] = dT
    du[2] = dx
end

In [ ]:
u0 = [70.0, 0.593]
tspan = (0.0, 50.0)
#p = (50.0, CTA.VelRampConst(10.0, 20.0), η, 0.01, 0.008)
vel = CTA.VelFreq(10.0, 0.5, 2)
p = (70.0, vel, η, 0.01, 0.008)

prob2 = ODEProblem(controle2, u0, tspan, p)
s = solve(prob2, dtmax=0.1)
T = s[1,:]
x = s[2,:]
Eo = efun.(x, Rt.(T))
#plot(s.t, s[1,:]./50.0)
plot(s.t, s[2,:])


In [ ]:
vel = CTA.VelRampConst(10.0, 10.0)
plot(s.t, vel.)


In [ ]:
xfun(10)